# Model Tutorial

In [1]:
import qnn_gen as qg
import numpy as np

"Model" is one of the most overloaded words in mathematical fields. One could consider a quantum machine learning "model" to mean the end-to-end processes of encoding, choice of gate architecture and parameters, measurement, post-processing, and optimization method. 

In QNN-Gen, the `Model` class is more restricted. The `Model` class corresponds to various achitectures of gates and ansätze with learnable parameters. Typically, these layers act on a state that has already been encoded. 

## The Model base class

In `model.py` you can find the `Model` base class and serveral implemenatations which derive from the base class. You can see that the derived classes ought to have attributes for `n_qubits` and `parameters`, and must override the `circuit` function. 

Additionally, the derived classes can override the `default_measurement` function. It is often the case that the choice of model implies which measurements are sensible. Implementing this function allows you to combine and run circuits without explictly passing a `Measurement` object.

In [2]:
from abc import ABC, abstractmethod

class Model(ABC):
    """
    Class to define a QML model, whether it be a particular ansatz or a perceptron model.

    Abstract class. Derived classes overwrite circuit() and may overwrite default_measuremnt().
    Derived classes should ensure that the self.n_qubits and self.parameters attributes are updated.
    """

    def __init__(self):
        self.n_qubits = None
        self.parameters = None

    @abstractmethod
    def circuit():
        """
        Abstract method. Overwrite to return the circuit.

        Returns:
            - (qiskit.QuantumCircuit): The circuit defined by the model
        """
        pass
    
    def default_measurement():
        """
        Often, the model implies what measurements are sensible.

        Returns:
            - (Measurement): The default measurement for the model
        """

        raise NotImplementedError

## Model architectures 
Now we'll go over some of the derived classes:
- Tree Tensor Network
- Binary Perceptron
- Entangled Qubit

### Tree Tensor Network

A simple way to instantiate a Tree Tensor network:

In [3]:
ttn = qg.TreeTensorNetwork(n_qubits=8)
ttn_circuit = ttn.circuit()

print(ttn_circuit)
print(ttn.parameters)

None
      ┌────────────┐                                                          
q_0: ─┤ RY(1.6693) ├──■───────────────────────────────────────────────────────
     ┌┴────────────┤┌─┴─┐ ┌────────────┐                                      
q_1: ┤ RY(0.34823) ├┤ X ├─┤ RY(2.9183) ├──■───────────────────────────────────
     ├─────────────┤└───┘ └────────────┘  │                                   
q_2: ┤ RY(0.79874) ├──■───────────────────┼───────────────────────────────────
     └┬────────────┤┌─┴─┐ ┌────────────┐┌─┴─┐┌────────────┐                   
q_3: ─┤ RY(1.0261) ├┤ X ├─┤ RY(2.6728) ├┤ X ├┤ RY(2.6916) ├──■────────────────
      ├───────────┬┘├───┤┌┴────────────┤├───┤├────────────┤┌─┴─┐┌────────────┐
q_4: ─┤ RY(1.666) ├─┤ X ├┤ RY(0.46207) ├┤ X ├┤ RY(1.9797) ├┤ X ├┤ RY(1.2324) ├
      ├───────────┴┐└─┬─┘└─────────────┘└─┬─┘└────────────┘└───┘└────────────┘
q_5: ─┤ RY(2.7564) ├──■───────────────────┼───────────────────────────────────
      └┬──────────┬┘┌───┐ ┌────────────┐  │    

If you don't pass arugments for the parameters of the rotation gates, they are randomly initialized. You may specificy them explictly using the `angles` argument:

In [4]:
angles = np.arange(0, 15)

ttn1 = qg.TreeTensorNetwork(n_qubits=8, angles=angles)
ttn_circuit1 = ttn1.circuit()

print(ttn_circuit1)
print(ttn1.parameters)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
     ┌───────┐                                             
q_0: ┤ RY(0) ├──■──────────────────────────────────────────
     ├───────┤┌─┴─┐┌───────┐                               
q_1: ┤ RY(1) ├┤ X ├┤ RY(8) ├───■───────────────────────────
     ├───────┤└───┘└───────┘   │                           
q_2: ┤ RY(2) ├──■──────────────┼───────────────────────────
     ├───────┤┌─┴─┐┌───────┐ ┌─┴─┐┌────────┐               
q_3: ┤ RY(3) ├┤ X ├┤ RY(9) ├─┤ X ├┤ RY(12) ├──■────────────
     ├───────┤├───┤├───────┴┐├───┤├────────┤┌─┴─┐┌────────┐
q_4: ┤ RY(4) ├┤ X ├┤ RY(10) ├┤ X ├┤ RY(13) ├┤ X ├┤ RY(14) ├
     ├───────┤└─┬─┘└────────┘└─┬─┘└────────┘└───┘└────────┘
q_5: ┤ RY(5) ├──■──────────────┼───────────────────────────
     ├───────┤┌───┐┌────────┐  │                           
q_6: ┤ RY(6) ├┤ X ├┤ RY(11) ├──■───────────────────────────
     ├───────┤└─┬─┘└────────┘                              
q_7: ┤ RY(7) ├──■────────────────────────────────────

User also may change the rotation gate and the entangling gate from their default values of `RY` and `CX`. For example, with the help of the `Gate` class in `utility.py`, a Tree Tensor Network with `RX` as the roation gate and a `CZ` as the two qubit gate:

In [5]:
ttn = qg.TreeTensorNetwork(n_qubits=8,
                           angles=angles, 
                           rotation_gate=qg.Gate.RX, 
                           entangling_gate=qg.Gate.CZ)
ttn_circuit = ttn.circuit()

print(ttn_circuit)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
     ┌───────┐                                       
q_0: ┤ RX(0) ├─■─────────────────────────────────────
     ├───────┤ │ ┌───────┐                           
q_1: ┤ RX(1) ├─■─┤ RX(8) ├──■────────────────────────
     ├───────┤   └───────┘  │                        
q_2: ┤ RX(2) ├─■────────────┼────────────────────────
     ├───────┤ │ ┌───────┐  │ ┌────────┐             
q_3: ┤ RX(3) ├─■─┤ RX(9) ├──■─┤ RX(12) ├─■───────────
     ├───────┤   ├───────┴┐   ├────────┤ │ ┌────────┐
q_4: ┤ RX(4) ├─■─┤ RX(10) ├─■─┤ RX(13) ├─■─┤ RX(14) ├
     ├───────┤ │ └────────┘ │ └────────┘   └────────┘
q_5: ┤ RX(5) ├─■────────────┼────────────────────────
     ├───────┤   ┌────────┐ │                        
q_6: ┤ RX(6) ├─■─┤ RX(11) ├─■────────────────────────
     ├───────┤ │ └────────┘                          
q_7: ┤ RX(7) ├─■─────────────────────────────────────
     └───────┘                                       


### Binary Perceptron

This is an example of a perceptron-based model. It encodes binary input and weight vectors, $\vec{i}$, $\vec{w}$ $\in \{1, −1\}^{2^N}$ into a quantum state of N qubits. The learning objective is to construct the sequence of gates which encodes the right weight vector for the binary function you wish to learn.

In QNN-Gen we base this algorithm using the sign-flip method which is defined defined
in [*] in the following steps:

1. Apply a full layer of H gates on all N qubits, each in the starting state $|0\rangle$.

2. Use the sign-flip method to encode the input $\vec{i}$

3. Use the sign-flip method to encode the weight vectors $\vec{w}$.

4. Apply a full layer of $H$ gates.

5. Apply a full layer of $X$ gates.

6. Apply a $C^X$ operation with the target on an ancillary qubit.

The Binary Peceptron model generates the circuit that enacts step 3 and on.

In [6]:
bp = qg.BinaryPerceptron(n_qubits=4)
bp_circuit = bp.circuit()

print(bp_circuit)
print(bp.parameters)

     ┌───┐   ┌───┐┌───┐   ┌───┐        ┌───┐        ┌───┐        ┌───┐        »
q_0: ┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├──────■─┤ X ├──────■─┤ X ├──────■─┤ X ├──────■─»
     ├───┤ │ ├───┤└───┘ │ ├───┤      │ ├───┤┌───┐ │ ├───┤┌───┐ │ ├───┤┌───┐ │ »
q_1: ┤ X ├─■─┤ X ├──────■─┤ X ├──────■─┤ X ├┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├┤ X ├─■─»
     ├───┤ │ ├───┤┌───┐ │ ├───┤      │ ├───┤└───┘ │ ├───┤├───┤ │ ├───┤└───┘ │ »
q_2: ┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├──────■─┤ X ├──────■─┤ X ├┤ X ├─■─┤ X ├──────■─»
     ├───┤ │ ├───┤├───┤ │ ├───┤┌───┐ │ ├───┤      │ └───┘└───┘ │ └───┘      │ »
q_3: ┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├──────■────────────■────────────■─»
     └───┘   └───┘└───┘   └───┘└───┘   └───┘                                  »
q_4: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«     ┌───┐        ┌───┐┌───┐          
«q_0: ┤ X ├──────■─┤ H ├┤ X ├───────■──
«     ├───┤┌───┐ │ ├───┤├───┤┌───┐  │  


Again, you may specify the starting weights explictly.

In [7]:
w = np.array([1, -1])
bp1 = qg.BinaryPerceptron(n_qubits=2, weights=w)
bp_circuit1 = bp1.circuit()

print(bp_circuit1)
print(bp1.parameters)

             ┌───┐┌───┐          
q_0: ──────■─┤ H ├┤ X ├───────■──
     ┌───┐ │ ├───┤├───┤┌───┐  │  
q_1: ┤ X ├─■─┤ X ├┤ H ├┤ X ├──■──
     └───┘   └───┘└───┘└───┘┌─┴─┐
q_2: ───────────────────────┤ X ├
                            └───┘
[ 1 -1]


### Entangled Qubit

This model comes from [*] and was used by the authors to classify MNIST data. The idea is that two-qubit gates connect a particular qubit to be measured with all other qubits.

In [8]:
et = qg.EntangledQubit(n_qubits=5)
et_circuit = et.circuit()

print(et_circuit)
print(et.parameters)

     ┌──────────────┐┌──────────────┐┌──────────────┐┌──────────────┐»
q_0: ┤0             ├┤0             ├┤0             ├┤0             ├»
     │  RXX(2.1182) ││              ││              ││              │»
q_1: ┤1             ├┤  RXX(6.2813) ├┤              ├┤              ├»
     └──────────────┘│              ││  RXX(1.9713) ││              │»
q_2: ────────────────┤1             ├┤              ├┤  RXX(1.0888) ├»
                     └──────────────┘│              ││              │»
q_3: ────────────────────────────────┤1             ├┤              ├»
                                     └──────────────┘│              │»
q_4: ────────────────────────────────────────────────┤1             ├»
                                                     └──────────────┘»
«     ┌──────────────┐┌──────────────┐┌───────────────┐┌───────────────┐
«q_0: ┤0             ├┤0             ├┤0              ├┤0              ├
«     │  RZX(1.9164) ││              ││               ││               │


In the entangled qubit model you can specify the gate set and the number of layers. The default `gate_set` is [`RXX`, `RZX`], and the default number of layers, `n_layers`, is $2$. The algorithm loops through the `gate_set` for the specified number of layers, repeating at the beginning if `n_layers` is greater than the number of gates in the gate set.


Here's an example with $3$ qubits, a gate set of [`RYY`, `RZZ`], and with $3$ layers.

In [9]:
et1 = qg.EntangledQubit(n_qubits=3, n_layers=3, gate_set=[qg.Gate.RYY, qg.Gate.RZZ])
et_circuit1 = et1.circuit()

print(et_circuit1)
print(et1.parameters)

     ┌───────────────┐┌──────────────┐                           »
q_0: ┤0              ├┤0             ├─■────────────■────────────»
     │  RYY(0.70385) ││              │ │zz(4.4103)  │            »
q_1: ┤1              ├┤  RYY(3.7201) ├─■────────────┼────────────»
     └───────────────┘│              │              │zz(0.25103) »
q_2: ─────────────────┤1             ├──────────────■────────────»
                      └──────────────┘                           »
«     ┌──────────────┐┌──────────────┐
«q_0: ┤0             ├┤0             ├
«     │  RYY(1.8325) ││              │
«q_1: ┤1             ├┤  RYY(2.6274) ├
«     └──────────────┘│              │
«q_2: ────────────────┤1             ├
«                     └──────────────┘
[0.70384597 3.72011248 4.41033538 0.25102793 1.83246804 2.62739117]


## Default Measurements

Each of the derived classes implement the function `default_measurement()`. We'll go over these measurement objects in the "Measurement and Observables" tutorial.

In [10]:
print(ttn.default_measurement())
print(bp.default_measurement())
print(et.default_measurement())